# main.py Explanation Notebook

## Importing Necessary Libraries

In [ ]:
import os
from pathlib import Path
from typing import List
from syftbox.lib import Client, SyftPermission
from pydantic import BaseModel
from pydantic_core import from_json
import logging

logger = logging.getLogger(__name__)

## Constants

In [ ]:
RING_APP_PATH = Path(os.path.abspath(__file__)).parent
CUSTOM_RING_APP_NAME = "ringapp"

## RingData Class

The `RingData` class is a Pydantic model that represents the data structure used in the ring.

In [ ]:
class RingData(BaseModel):
    ring: list[str]
    data: int
    current_index: int

    @property
    def ring_length(self) -> int:
        return len(self.ring)

    @classmethod
    def load_json(cls, file):
        with open(file, "r") as f:
            return cls(**from_json(f.read()))

### Explanation:
- `ring`: The list of email addresses in the ring, that is the list of datasites attending the ring app.
- `data`: An integer representing the data being processed.
- `current_index`: The current index in the ring.
- `ring_length`: A property that returns the length of the ring.
- `load_json`: A class method to load `RingData` from a JSON file.

Indeed, here's an example of the `data.json` file before running the app:
```json
{
    "ring": ["client1@email.com", "client2@email.com", "client3@email.com"],
    "data": 0,
    "current_index": 0
}
```

## RingRunner Class

The `RingRunner` class handles the main logic of processing the ring data.

In [ ]:
class RingRunner:
    def __init__(self):
        self.client = Client.load()
        self.my_email: str = self.client.email
        self.ring_pipeline_path: Path = Path(self.client.datasite_path) / "app_pipelines" / CUSTOM_RING_APP_NAME
        self.running_folder: Path = self.ring_pipeline_path / "running"
        self.done_folder: Path = self.ring_pipeline_path / "done"
        self.secret_filename: Path = "secret.txt"

### Explanation:
- `__init__`: Initializes the `RingRunner` with paths and client information.

### Methods

#### run

In [ ]:
    def run(self) -> None:
        self.setup_folders()
        input_files = self.pending_inputs_files()
        for file_name in input_files:
            self.process_input(file_name)

        if len(input_files) == 0:
            print("No data file found. As you were, soldier.")

It is designed to manage the main workflow of processing ring data. Here's the steps:

1. It first ensures that the necessary directories (`running` and `done`) are created and ready for use, by calling `self.setup_folders()` method.
2. It retrieves a list of pending input files (JSON files) that need to be processed, by calling `self.pending_inputs_files()` method.
3. It iterates over each file in the list of pending input files and processes them one by one, by calling ` self.process_input(file_name)` method. 
4. If no input files are found (the list is empty), it prints a message indicating that no data files were found and there is nothing to process.

#### process_input

In [ ]:
    def process_input(self, file_path) -> None:
        print(f"Found input {file_path}! Let's get to work.")
        logger.debug(file_path)
        ring_data = RingData.load_json(file_path)

        actual_email = next_person = ring_data.ring[ring_data.current_index]
        ring_data.data += self.my_secret(actual_email)
        next_index = ring_data.current_index + 1

        if next_index < ring_data.ring_length:
            next_person = ring_data.ring[next_index]
            ring_data.current_index = next_index
            self.send_data(next_person, ring_data)
        else:
            self.terminate_ring(ring_data)

        self.cleanup(file_path)

#### cleanup

In [ ]:
    def cleanup(self, file_path: Path) -> None:
        file_path.unlink()
        print(f"Done processing {file_path}, removed from pending inputs")

#### setup_folders

In [ ]:
    def setup_folders(self) -> None:
        print("Setting up the necessary folders.")
        for folder in [self.running_folder, self.done_folder]:
            folder.mkdir(parents=True, exist_ok=True)

        permission = SyftPermission.mine_with_public_write(self.my_email)
        permission.ensure(self.ring_pipeline_path)

#### my_secret

In [ ]:
    def my_secret(self, email: str = None):
        path_secret_file = Path(self.client.sync_folder) / email / "privatefolder" / self.secret_filename
        with open(path_secret_file, "r") as secret_file:
            return int(secret_file.read().strip())

This method retrieves a secret value associated with a given email. 

#### pending_inputs_files

In [ ]:
    def pending_inputs_files(self) -> List[Path]:
        return [self.running_folder / file for file in self.running_folder.glob("*.json")]

#### write_json

In [ ]:
    def write_json(self, file_path: Path, result: RingData) -> None:
        print(f"Writing to {file_path}.")
        file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(file_path, "w") as f:
            f.write(result.model_dump_json())

#### send_data

In [ ]:
    def send_data(self, email: str, data: RingData) -> None:
        destination_datasite_path = Path(self.client.sync_folder) / email
        dest = destination_datasite_path / "app_pipelines" / CUSTOM_RING_APP_NAME / "running" / "data.json"
        self.write_json(dest, data)

It's responsible for sending processed ring data to the next person in the ring.

#### terminate_ring

In [ ]:
    def terminate_ring(self, data: RingData) -> None:
        print(f"Terminating ring, writing back to {self.done_folder}")
        self.write_json(self.done_folder / "data.json", data)

## Main Function

In [ ]:
if __name__ == "__main__":
    runner = RingRunner()
    runner.run()

### Explanation:
- The main function creates an instance of `RingRunner` and calls the `run` method to start the process.